# **CIANNA Stellar Spectra**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/AI_astro_ED_AAIF/blob/main/practical_works/CNN/classification/spectra/Spectra_CIANNA.ipynb)

---


**Link to the CIANNA github repository**
https://github.com/Deyht/CIANNA

### **CIANNA installation**

#### Query GPU allocation and properties

If nvidia-smi fail, it might indicate that you launched the colab session whithout GPU reservation.  
To change the type of reservation go to "Runtime"->"Change runtime type" and select "GPU" as your hardware accelerator.

In [ ]:
%%shell

nvidia-smi

cd /content/

git clone https://github.com/NVIDIA/cuda-samples/

cd /content/cuda-samples/Samples/1_Utilities/deviceQuery/

cmake CMakeLists.txt

make SMS="50 60 70 80"

./deviceQuery | grep Capability | cut -c50- > ~/cuda_infos.txt
./deviceQuery | grep "CUDA Driver Version / Runtime Version" | cut -c57- >> ~/cuda_infos.txt

cd ~/

If you are granted a GPU that supports high FP16 compute scaling (e.g the Tesla T4), it is advised to change the mixed_precision parameter in the last cell to "FP16C_FP32A".  
See the detail description on mixed precision support with CIANNA on the [Systeme Requirements](https://github.com/Deyht/CIANNA/wiki/1\)-System-Requirements) wiki page.

#### Clone CIANNA git repository

In [ ]:
%%shell

cd /content/

git clone https://github.com/Deyht/CIANNA

cd CIANNA

#### Compiling CIANNA for the allocated GPU generation

There is no guaranteed forward or backward compatibility between Nvidia GPU generation, and some capabilities are generation specific. For these reasons, CIANNA must be provided the platform GPU generation at compile time.
The following cell will automatically update all the necessary files based on the detected GPU, and compile CIANNA.

In [ ]:
%%shell

cd /content/CIANNA

mult="10"
cat ~/cuda_infos.txt
comp_cap="$(sed '1!d' ~/cuda_infos.txt)"
cuda_vers="$(sed '2!d' ~/cuda_infos.txt)"

lim="11.1"
old_arg=$(awk '{if ($1 < $2) print "-D CUDA_OLD";}' <<<"${cuda_vers} ${lim}")

sm_val=$(awk '{print $1*$2}' <<<"${mult} ${comp_cap}")

gen_val=$(awk '{if ($1 >= 80) print "-D GEN_AMPERE"; else if($1 >= 70) print "-D GEN_VOLTA";}' <<<"${sm_val}")

sed -i "s/.*arch=sm.*/\\t\tcuda_arg=\"\$cuda_arg -D CUDA -D comp_CUDA -lcublas -lcudart -arch=sm_$sm_val $old_arg $gen_val\"/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" src/python_module_setup.py

./compile.cp CUDA PY_INTERF

mv src/build/lib.linux-x86_64-* src/build/lib.linux-x86_64

#### Testing CIANNA installation

**IMPORTANT NOTE**   
CIANNA is mainly used in a script fashion and was not designed to run in notebooks. Every cell code that directly invokes CIANNA functions must be run as a script to avoid possible errors.  
To do so, the cell must have the following structure.

```
%%shell

cd /content/CIANNA

python3 - <<EOF

[... your python code ...]

EOF
```

This syntax allows one to easily edit python code in the notebook while running the cell as a script. Note that all the notebook variables can not be accessed by the cell in this context.


In [ ]:
%%shell

cd /content/

python3 - <<EOF


import numpy as np
import matplotlib.pyplot as plt
import os

import sys, glob
sys.path.insert(0,glob.glob('/content/CIANNA/src/build/lib.*/')[-1])
import CIANNA as cnn

############################################################################
##              Data reading (your mileage may vary)
############################################################################

def i_ar(int_list):
	return np.array(int_list, dtype="int")

def f_ar(float_list):
	return np.array(float_list, dtype="float32")

if(not os.path.isdir("stellar_spectra_data")):
		os.system("wget https://share.obspm.fr/s/ANxKkxAZoKmXzRw/download/stellar_spectra_data.tar.gz")
		os.system("tar -xzf stellar_spectra_data.tar.gz")

print ("Reading inputs ... ", end = "", flush=True)

nb_class = 7
raw_spectra_size = 3753
spectra_size = 3776
nb_spectra = 1115
nb_keep_val = 200

class_balance = [10,30,30,20,20,30,10]
nb_train = np.sum(class_balance) # rebalanced over 1115

######################### ##########################
#          Loading data and pre process
######################### ##########################

raw_data = np.loadtxt("stellar_spectra_data/train.dat")
raw_target = np.loadtxt("stellar_spectra_data/target.dat")

input = np.tanh(40*np.clip(raw_data,0.,1.))
targ = raw_target

id_classes = []
for i in range(0, nb_class):
	temp = np.where(np.argmax(targ[:,:], axis=1) == i)
	id_classes.append(temp)

# split training and test dataset
input_test = input[-nb_keep_val:,:]
targ_test = targ[-nb_keep_val:,:]

print (np.shape(input_test))

input_train = np.empty((0,raw_spectra_size))
targ_train = np.empty((0,nb_class))

for i in range(0,nb_class):
	index_list = id_classes[i][0][:class_balance[i]]
	for j in range(0, len(index_list)):
		input_train = np.append(input_train, np.reshape(input[index_list[j],:],(1,raw_spectra_size)), axis=0)
		targ_train = np.append(targ_train, np.reshape(targ[index_list[j],:],(1,nb_class)), axis=0)

pad_in_train = np.zeros((nb_train, spectra_size))
pad_in_test = np.zeros((nb_keep_val, spectra_size))

pre_pad = np.maximum(0,(spectra_size - raw_spectra_size)//2)
post_pad = np.maximum(0,(spectra_size - raw_spectra_size)//2 + (spectra_size - raw_spectra_size)%2)

pad_in_train[:,pre_pad:-post_pad] = input_train[:,:]
pad_in_test[:,pre_pad:-post_pad] = input_test[:,:]

############################################################################
##               CIANNA network construction and use
############################################################################

#Details about the functions and parameters are given in the GitHub Wiki

cnn.init(in_dim=i_ar([spectra_size]), in_nb_ch=1, out_dim=7,
		bias=0.1, b_size=16, comp_meth="C_CUDA", #Change to C_BLAS or C_NAIV
		dynamic_load=1, mixed_precision="FP32C_FP32A")

cnn.create_dataset("TRAIN", size=nb_train   , input=f_ar(pad_in_train), target=f_ar(targ_train))
cnn.create_dataset("VALID", size=nb_keep_val, input=f_ar(pad_in_test) , target=f_ar(targ_test))
cnn.create_dataset("TEST" , size=nb_keep_val, input=f_ar(pad_in_test) , target=f_ar(targ_test))

#Python side datasets are not required anymore, they can be released to save RAM

#Used to load a saved network at a given iteration
load_step = 0
if(load_step > 0):
	cnn.load("net_save/net0_s%04d.dat"%(load_step), load_step)
else:
	cnn.conv(f_size=i_ar([8]), nb_filters=8 , stride=i_ar([4]), padding=i_ar([2]), activation="RELU")
	cnn.conv(f_size=i_ar([8]), nb_filters=16, stride=i_ar([4]), padding=i_ar([2]), activation="RELU")
	cnn.conv(f_size=i_ar([8]), nb_filters=32, stride=i_ar([4]), padding=i_ar([2]), activation="RELU")
	cnn.dense(nb_neurons=256, drop_rate=0.4, activation="RELU")
	cnn.dense(nb_neurons=256, drop_rate=0.4, activation="RELU")
	cnn.dense(nb_neurons=nb_class, strict_size=1, activation="SMAX")

cnn.train(nb_iter=800, learning_rate=0.002, momentum=0.8, confmat=1, weight_decay=0.001, control_interv=20, save_every=0)
cnn.perf_eval()

#Uncomment to save network prediction
#cnn.forward(repeat=1, drop_mode="AVG_MODEL")

EOF